# Imports

There is also a file named `requirements.txt`, which will install all dependencies required for training to work.

Note: Mask-RCNN does **NOT** work with Tensorflow 2. This notebook has been run with:

- Python 3.7
- Tensorflow 1.15.0 (or tensorflow-gpu)
- Keras 2.4.5

In [1]:
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf

# Comment out everything below if you are NOT using a GPU for training.
# Feel free to change the config.gpu_options.per_process_gpu_memory_fraction to an appropriate percentage
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# tf.Session(config=config)

Note: The directory can also be concatenated using `os.path.join()` in the event you don't want to mess with the `\` convention for Windows and the universal `/` convention for every other OS. Also note that using `\` in syntax requires you to write it out as `\\` because this indicates the insert of a special character (i.e `\n` means new line).

If you're on Windows, and you're using `os.path.join`, it's a good idea to leave the directory with a `\\` so that all the occurrences of `os.path.join()` will know that a `\` convention is used.

In [3]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = 'C:\\Users\\iFai1\\Desktop\\Cornell\\MRCNN_Iteration\\Mask_RCNN\\'

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import samples.spine_segmented.spine_segmented as spine

%matplotlib inline 

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


# Training

There is another notebook that goes through training independent of inputting into the command line, such as the command below. That notebook in particular will give a better understanding of how training works.

For simplicity in this notebook, we simply call the command below to launch training.

## Reference of Commands
For reference, this is a copy and paste of of training commands from the Mask-RCNN README:

We're providing pre-trained weights for MS COCO to make it easier to start. You can
use those weights as a starting point to train your own variation on the network.
Training and evaluation code is in `samples/coco/coco.py`. You can import this
module in Jupyter notebook (see the provided notebooks for examples) or you
can run it directly from the command line as such:

```
# Train a new model starting from pre-trained COCO weights
python3 samples/coco/coco.py train --dataset=/path/to/coco/ --model=coco

# Train a new model starting from ImageNet weights
python3 samples/coco/coco.py train --dataset=/path/to/coco/ --model=imagenet

# Continue training a model that you had trained earlier
python3 samples/coco/coco.py train --dataset=/path/to/coco/ --model=/path/to/weights.h5

# Continue training the last model you trained. This will find
# the last trained weights in the model directory.
python3 samples/coco/coco.py train --dataset=/path/to/coco/ --model=last
```

You can also run the COCO evaluation code with:
```
# Run COCO evaluation on the last trained model
python3 samples/coco/coco.py evaluate --dataset=/path/to/coco/ --model=last
```

## Troubleshooting

Here are some common problems I have came across when trying to get this to run on my computer. Refer to these if you're stuck.

1. There's an error indicating directories weren't found, or some of the parameters (i.e --weights="coco") are not valid.
    - Check to make sure directory paths are valid. Also, Windows convention (not surprised) must use `"` instead of `'` when inputting strings/parameters. For example, saying `--weights='coco'` on Windows will import it as `'coco'`, and Mask-RCNN is just checking for `coco`.
    
    
2. `Resource exhausted: OOM when allocating tensor with shape[128,14,14,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc`
    - There's not enough VRAM on your GPU. This could be of two causes. Check to see if you have other datasets/Jupyter notebooks loaded and running on your card. Be sure to shut them off. If you are still experiencing this problem, your `config.gpu_options.per_process_gpu_memory_fraction` config in the beginning of the notebook (when importing Tensorflow) might be set too high.
    - Also, if you have ran this notebook multiple times, you should restart the kernel to clear any old datasets and imports that have been loaded onto the card. Then, run this again.

3. `ValueError: need at least one array to stack`
    - This is most likely caused by an image that doesn't have **at least** one mask for the image. Causing an error. Make sure to clean up your dataset if that is the case! There should be some scripts in `/data_utils` that I have created that will do this.

In [4]:
!python C:/Users/iFai1/Desktop/Cornell/MRCNN_Iteration/Mask_RCNN/samples/spine_segmented/spine_segmented.py train --dataset="C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\datasets\spine_segmented" --subset=train --weights="coco"

Weights:  coco
Dataset:  C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\datasets\spine_segmented
Subset:  train
Logs:  C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS      

2020-06-07 17:42:12.168427: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


2020-06-07 17:42:13.243403: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-06-07 17:42:13.246074: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-06-07 17:42:13.265871: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2070 with Max-Q Design major: 7 minor: 5 memoryClockRate(GHz): 1.185
pciBusID: 0000:01:00.0
2020-06-07 17:42:13.266077: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll
2020-06-07 17:42:13.268032: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cublas64_100.dll
2020-06-07 17:42:13.269793

VALIDATION_STEPS               5
WEIGHT_DECAY                   0.0001


Loading weights  C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\mask_rcnn_coco.h5
Train network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\logs\spine20200607T1742\mask_rcnn_spine_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistribute


ValueError: need at least one array to stack
Traceback (most recent call last):
  File "C:/Users/iFai1/Desktop/Cornell/MRCNN_Iteration/Mask_RCNN/samples/spine_segmented/spine_segmented.py", line 490, in <module>
    train(model, args.dataset, args.subset)
  File "C:/Users/iFai1/Desktop/Cornell/MRCNN_Iteration/Mask_RCNN/samples/spine_segmented/spine_segmented.py", line 291, in train
    layers='heads')
  File "C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\mrcnn\model.py", line 2376, in train
    use_multiprocessing=True,
  File "C:\ProgramData\Anaconda3\envs\cornell-mrcnn\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\cornell-mrcnn\lib\site-packages\keras\engine\training.py", line 1658, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\ProgramData\Anaconda3\envs\cornell-mrcnn\lib\site-packages\keras\engine\training_generator.py", line 181, in fit_generator
    generator_outp

# Setup our model for inference

The following cell is just some prelimnary formatting to display images on the notebook. Change as you see fit for the image to be displayed in this notebook. This does not interfere with anything pertaining to our training/model.

In [7]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    fig.tight_layout()
    return ax

The following to check out parameters of the model for double checking and reference.

In [5]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, "datasets/spine_segmented")

# Inference Configuration
config = spine.SpineInferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [8]:
# Load validation dataset
dataset = spine.SpineDataset()
dataset.load_spine(DATASET_DIR, "train") # train is selected here on purpose to test on train set first
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 1910
Classes: ['BG', 'v1', 'v2', 'v3']


# Load model & weights

In [6]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# Only inference mode is supported right now
TEST_MODE = "inference"

In [9]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE,
                              model_dir=LOGS_DIR,
                              config=config)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [10]:
# Path to a specific weights file
# weights_path = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Or, load the last model you trained
weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)
# model.load_weights(weights_path, by_name=True, exclude=[ "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

Loading weights  C:\Users\iFai1\Desktop\Cornell\MRCNN_Iteration\Mask_RCNN\logs\spine20200607T1545\mask_rcnn_spine_0001.h5


OSError: Unable to open file (bad object header version number)

# Validation

If you are experiencing errors with this, such as the `ValueError: need at least one array to stack` error, the image chosen might not have detected anything at all, causing this error.

To insert more images and their ground truths for testing, place them into `MaskRCNN/datasets/spine_segmented/data/val` and place them either in `/img` or `/gt` whether they're the physical image or the ground truth.

You can rerun this again to receive another image. If there is a particular image you wish you see, simply edit `image_id`.

In [ ]:
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))
print("Original image shape: ", modellib.parse_image_meta(image_meta[np.newaxis,...])["original_image_shape"][0])

# Run object detection
results = model.detect_molded(np.expand_dims(image, 0), np.expand_dims(image_meta, 0), verbose=1)

# Display results
r = results[0]
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Compute AP over range 0.5 to 0.95 and print it
utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                       r['rois'], r['class_ids'], r['scores'], r['masks'],
                       verbose=1)

visualize.display_differences(
    image,
    gt_bbox, gt_class_id, gt_mask,
    r['rois'], r['class_ids'], r['scores'], r['masks'],
    dataset.class_names, ax=get_ax(),
    show_box=False, show_mask=False,
    iou_threshold=0.5, score_threshold=0.5)